In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Chargement des données
data = pd.read_csv('df_merged_bretagne_filtre.csv', sep=';', decimal=',')

# Conversion des colonnes numériques
numeric_cols = ['Votants', '% Votants', '%_voix_obtenu',
                '0 à 19 ans', '20 à 39 ans', '40 à 59 ans', '60 à 74 ans', '75 ans et plus',
                'Total', 'taux_chomage', 'taux_pour_mille', 'nombre_entreprises',
                'Baccalauréat professionnel', 'Baccalauréat technologique', 'Baccalauréat général']

for col in numeric_cols:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Création de nouvelles colonnes pour l'analyse
data['% 0-19 ans'] = data['0 à 19 ans'] / data['Total'] * 100
data['% 20-39 ans'] = data['20 à 39 ans'] / data['Total'] * 100
data['% 40-59 ans'] = data['40 à 59 ans'] / data['Total'] * 100
data['% 60-74 ans'] = data['60 à 74 ans'] / data['Total'] * 100
data['% 75+ ans'] = data['75 ans et plus'] / data['Total'] * 100

# Regroupement par année et orientation politique
data_by_year = data.groupby('Année').first().reset_index()
data_left = data[data['COULEUR POLITIQUE'].isin(['Gauche', 'Centre, progressiste'])]
data_right = data[data['COULEUR POLITIQUE'].isin(['Droite républicaine', 'Extrême droite'])]

# Visualisation 1 : Évolution de la participation électorale
fig1 = px.line(data_by_year, x='Année', y='% Votants',
              title='Évolution de la Participation Électorale en Bretagne (1995-2022)',
              markers=True)
fig1.update_layout(xaxis_title='Année', yaxis_title='Pourcentage de Participation (%)')
fig1.update_yaxes(range=[75, 90])

# Visualisation 2 : Résultats électoraux par orientation politique
fig2 = go.Figure()

# Extraction des données pour la gauche/centre-progressiste
left_years = data_left['Année'].tolist()
left_votes = data_left['%_voix_obtenu'].tolist()
left_candidates = data_left['nom_prenom'].tolist()
left_parties = data_left['PARTI POLITIQUE'].tolist()
left_labels = [f"{y}: {c} ({p})" for y, c, p in zip(left_years, left_candidates, left_parties)]

# Extraction des données pour la droite/extrême droite
right_years = data_right['Année'].tolist()
right_votes = data_right['%_voix_obtenu'].tolist()
right_candidates = data_right['nom_prenom'].tolist()
right_parties = data_right['PARTI POLITIQUE'].tolist()
right_labels = [f"{y}: {c} ({p})" for y, c, p in zip(right_years, right_candidates, right_parties)]

# Ajout des barres pour la gauche/centre-progressiste
fig2.add_trace(go.Bar(
    x=left_years,
    y=left_votes,
    name='Gauche/Centre-Progressiste',
    marker_color='#FF9999',
    hovertemplate='%{y:.2f}% - %{text}<extra></extra>',
    text=[f"{c} ({p})" for c, p in zip(left_candidates, left_parties)]
))

# Ajout des barres pour la droite/extrême droite
fig2.add_trace(go.Bar(
    x=right_years,
    y=right_votes,
    name='Droite/Extrême Droite',
    marker_color='#99CCFF',
    hovertemplate='%{y:.2f}% - %{text}<extra></extra>',
    text=[f"{c} ({p})" for c, p in zip(right_candidates, right_parties)]
))

fig2.update_layout(
    title='Résultats Électoraux par Orientation Politique (1995-2022)',
    xaxis_title='Année',
    yaxis_title='Pourcentage de Voix (%)',
    barmode='group'
)

# Visualisation 3 : Évolution démographique
fig3 = go.Figure()

age_groups = ['% 0-19 ans', '% 20-39 ans', '% 40-59 ans', '% 60-74 ans', '% 75+ ans']
colors = ['#66C2A5', '#FC8D62', '#8DA0CB', '#E78AC3', '#A6D854']
labels = ['0-19 ans', '20-39 ans', '40-59 ans', '60-74 ans', '75+ ans']

for i, age_group in enumerate(age_groups):
    fig3.add_trace(go.Scatter(
        x=data_by_year['Année'],
        y=data_by_year[age_group],
        mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=2),
        marker=dict(size=8)
    ))

fig3.update_layout(
    title='Évolution de la Distribution par Âge en Bretagne (1995-2022)',
    xaxis_title='Année',
    yaxis_title='Pourcentage de Population (%)',
    legend_title='Groupe d\'Âge'
)

# Visualisation 4 : Corrélation entre taux de chômage et vote
fig4 = px.scatter(data, x='taux_chomage', y='%_voix_obtenu', color='COULEUR POLITIQUE',
                 size='Votants', size_max=30,
                 hover_data=['nom_prenom', 'Année', 'PARTI POLITIQUE'],
                 title='Corrélation entre Taux de Chômage et Pourcentage de Voix')

fig4.update_layout(
    xaxis_title='Taux de Chômage (%)',
    yaxis_title='Pourcentage de Voix (%)',
    legend_title='Orientation Politique'
)

# Visualisation 5 : Corrélation entre niveau d'éducation et vote
fig5 = make_subplots(rows=1, cols=3, subplot_titles=(
    'Baccalauréat Professionnel vs Voix',
    'Baccalauréat Technologique vs Voix',
    'Baccalauréat Général vs Voix'))

# Bac Pro
fig5.add_trace(
    go.Scatter(
        x=data['Baccalauréat professionnel'],
        y=data['%_voix_obtenu'],
        mode='markers',
        marker=dict(
            color=data['Année'],
            colorscale='Viridis',
            size=10,
            colorbar=dict(title='Année'),
            showscale=True
        ),
        text=data['nom_prenom'] + ' (' + data['Année'].astype(str) + ')',
        hoverinfo='text+x+y'
    ),
    row=1, col=1
)

# Bac Techno
fig5.add_trace(
    go.Scatter(
        x=data['Baccalauréat technologique'],
        y=data['%_voix_obtenu'],
        mode='markers',
        marker=dict(
            color=data['Année'],
            colorscale='Viridis',
            size=10,
            showscale=False
        ),
        text=data['nom_prenom'] + ' (' + data['Année'].astype(str) + ')',
        hoverinfo='text+x+y'
    ),
    row=1, col=2
)

# Bac General
fig5.add_trace(
    go.Scatter(
        x=data['Baccalauréat général'],
        y=data['%_voix_obtenu'],
        mode='markers',
        marker=dict(
            color=data['Année'],
            colorscale='Viridis',
            size=10,
            showscale=False
        ),
        text=data['nom_prenom'] + ' (' + data['Année'].astype(str) + ')',
        hoverinfo='text+x+y'
    ),
    row=1, col=3
)

fig5.update_layout(
    title_text='Corrélation entre Niveau d\'Éducation et Vote',
    height=500,
    width=1000
)

fig5.update_xaxes(title_text='% Baccalauréat Professionnel', row=1, col=1)
fig5.update_xaxes(title_text='% Baccalauréat Technologique', row=1, col=2)
fig5.update_xaxes(title_text='% Baccalauréat Général', row=1, col=3)

fig5.update_yaxes(title_text='% Voix', row=1, col=1)

# Visualisation 6 : Évolution des indicateurs socio-économiques
fig6 = make_subplots(specs=[[{"secondary_y": True}]])

fig6.add_trace(
    go.Scatter(
        x=data_by_year['Année'],
        y=data_by_year['taux_chomage'],
        mode='lines+markers',
        name='Taux de Chômage',
        line=dict(color='red', width=2),
        marker=dict(size=8)
    ),
    secondary_y=False
)

fig6.add_trace(
    go.Scatter(
        x=data_by_year['Année'],
        y=data_by_year['nombre_entreprises'],
        mode='lines+markers',
        name='Nombre d\'Entreprises',
        line=dict(color='blue', width=2),
        marker=dict(size=8)
    ),
    secondary_y=True
)

fig6.update_layout(
    title_text='Évolution des Indicateurs Socio-économiques en Bretagne (1995-2022)',
    xaxis_title='Année'
)

fig6.update_yaxes(title_text='Taux de Chômage (%)', secondary_y=False)
fig6.update_yaxes(title_text='Nombre d\'Entreprises', secondary_y=True)

# Visualisation 7 : Matrice de corrélation
# Calcul des corrélations entre variables numériques
corr_columns = ['% Votants', '%_voix_obtenu', 'taux_chomage', 'nombre_entreprises',
                'Baccalauréat professionnel', 'Baccalauréat technologique', 'Baccalauréat général',
                '% 0-19 ans', '% 20-39 ans', '% 40-59 ans', '% 60-74 ans', '% 75+ ans']

corr_data = data[corr_columns].corr()

fig7 = px.imshow(
    corr_data,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    aspect="auto",
    title='Matrice de Corrélation entre Variables'
)

fig7.update_layout(height=700, width=700)

# Affichage des visualisations
fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()
fig6.show()
fig7.show()

In [6]:
# Statistiques descriptives pour les variables numériques
print(data.describe())

# Statistiques descriptives pour les variables catégorielles
print(data.describe(include=['object']))

# Analyse de la distribution des votes par orientation politique
print(data.groupby('COULEUR POLITIQUE')['%_voix_obtenu'].describe())

# Analyse de la participation électorale par année
print(data.groupby('Année')['% Votants'].describe())

# Analyse du taux de chômage par orientation politique
print(data.groupby('COULEUR POLITIQUE')['taux_chomage'].describe())

# Analyse du niveau d'éducation par orientation politique
print(data.groupby('COULEUR POLITIQUE')[['Baccalauréat professionnel', 'Baccalauréat technologique', 'Baccalauréat général']].describe())

# Analyse de la structure démographique par orientation politique
print(data.groupby('COULEUR POLITIQUE')[['% 0-19 ans', '% 20-39 ans', '% 40-59 ans', '% 60-74 ans', '% 75+ ans']].describe())


             Année       Votants  % Votants  %_voix_obtenu     0 à 19 ans  \
count    12.000000  1.200000e+01  12.000000      12.000000      12.000000   
mean   2009.166667  1.924692e+06  82.653333      50.000000  764147.833333   
std       9.446821  1.144840e+05   3.309340      21.116329   21672.939285   
min    1995.000000  1.740335e+06  77.900000      11.440000  735549.000000   
25%    2002.000000  1.811287e+06  79.690000      41.092500  738672.000000   
50%    2009.500000  1.975712e+06  82.970000      50.000000  769482.500000   
75%    2017.000000  2.019910e+06  84.860000      58.907500  781997.000000   
max    2022.000000  2.025199e+06  87.530000      88.560000  789704.000000   

         20 à 39 ans    40 à 59 ans    60 à 74 ans  75 ans et plus  \
count      12.000000      12.000000      12.000000       12.000000   
mean   761214.333333  820689.500000  510729.833333   295447.000000   
std     29934.987336   81187.297035   87461.827410    65256.397828   
min    721830.000000  6652